In [1]:
import tensorflow as tf
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Fixes bad convolution
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
def deg2rad(deg):
    pi_on_180 = 0.017453292519943295
    return deg * pi_on_180

def rotation_matrix(theta):
    rotation_matrix = tf.stack([tf.cos(theta),
                              -tf.sin(theta),  
                               tf.sin(theta),
                               tf.cos(theta)])
    return tf.reshape(rotation_matrix, (2,2))

s = tf.io.read_file('day12.txt')
s = tf.strings.split(s, '\n')
directions = tf.strings.substr(s, 0, 1)
values = tf.strings.substr(s, 1, -1)
values = tf.strings.to_number(values)

@tf.function(experimental_compile=False)
def solve(directions, values):
    ship_dir = tf.constant([[1.0], [0.0]])
    ship_coords = tf.constant([[0.0], [0.0]])
    for i in tf.range(directions.shape[0]):
        direction = directions[i]
        val = values[i]

        if direction == 'N':
            ship_coords += val*tf.constant([0.0, 1.0])[:,None]
        elif direction == 'E':
            ship_coords += val*tf.constant([1.0, 0.0])[:,None]
        elif direction == 'S':
            ship_coords += val*tf.constant([0.0, -1.0])[:,None]
        elif direction == 'W':
            ship_coords += val*tf.constant([-1.0, 0.0])[:,None]
        elif direction == 'F':
            ship_coords += val*ship_dir
        elif direction == 'L':
            deg = deg2rad(val)
            ship_dir = tf.round(rotation_matrix(deg)@ship_dir)
        elif direction == 'R':
            deg = deg2rad(val)
            ship_dir = tf.round(rotation_matrix(-deg)@ship_dir)
    
    return tf.reduce_sum(tf.abs(ship_coords))

%timeit solve(directions, values)
solve(directions, values)

83.2 ms ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=1186.0>

In [3]:
@tf.function(experimental_compile=False)
def solve(directions, values):
    wp_coords = tf.constant([[10.0], [1.0]])
    ship_coords = tf.constant([[0.0], [0.0]])
    for i in tf.range(directions.shape[0]):
        direction = directions[i]
        val = values[i]

        if direction == 'N':
            wp_coords += val*tf.constant([0.0, 1.0])[:,None]
        elif direction == 'E':
            wp_coords += val*tf.constant([1.0, 0.0])[:,None]
        elif direction == 'S':
            wp_coords += val*tf.constant([0.0, -1.0])[:,None]
        elif direction == 'W':
            wp_coords += val*tf.constant([-1.0, 0.0])[:,None]
        elif direction == 'F':
            ship_coords += val*wp_coords
        elif direction == 'L':
            deg = deg2rad(val)
            wp_coords = tf.round(rotation_matrix(deg)@wp_coords)
        elif direction == 'R':
            deg = deg2rad(val)
            wp_coords = tf.round(rotation_matrix(-deg)@wp_coords)
    
    return tf.reduce_sum(tf.abs(ship_coords))

%timeit solve(directions, values)
solve(directions, values)

78.6 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=47806.0>